In [25]:
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
benchmark_file = "./profile_server.csv"

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv(benchmark_file)
num_rows, num_cols = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")
print("Header:", df.columns)

qps = "QPS"
throughput = "throughput(token/s)"

In [ ]:
# the 32th row
df.iloc[31]

# the rows from the 11th to the 20th
df.iloc[10:20]

# display the first 10 rows
df.head(n=10)


In [ ]:
# Get unique values
unique_text_length = df['text_length'].unique()
print("Unique values using unique():", unique_text_length)

# Get the number of unique values
unique_count = df['text_length'].nunique()
print("Number of unique values using nunique():", unique_count)

In [ ]:
#df["text_length"]
#df["QPS"].max()
# Find the row with the maximum QPS
max_qps_row = df.loc[df[qps].idxmax()]
# Print the row with the maximum QPS
print("Max QPS:")
print(max_qps_row)

max_throughtpus_row = df.loc[df[throughput].idxmax()]
print("Max throughput:")
print(max_throughtpus_row)


In [ ]:
# Sort the DataFrame by the 'QPS' column in descending order
df_by_qps = df.sort_values(by=qps, ascending=False)
df_by_qps.head(n=50)

In [ ]:
# Sort the DataFrame by the throughput column in ascending order
df_by_throughput = df.sort_values(by=throughput, ascending=False)
df_by_throughput.head(n=50)


In [ ]:
# Filter the DataFrame using multiple conditions
filtered_df = df[(df[qps] >= 800) & (df[throughput] >= 500000)]
print(filtered_df.shape)
# Print the filtered DataFrame
filtered_df

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))

# (batch_size, worker_num)
for t, w in [(128, 31), (256, 31), (512, 31), (1024, 31)]:
    filtered_df = df[(df["text_length"] == t) & (df["worker_num"] == w)]
    plt.plot(filtered_df['batch_size'], filtered_df[qps], marker='o', linestyle='-', label=f'text_length={t},worker_num={w}')

plt.title("Batch Size vs QPS")
plt.xlabel('Batch Size')
plt.ylabel(qps)
# Show the legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))

# (batch_size, worker_num)
for t, w in [(1, 31), (4, 31), (8, 31), (12, 31), (16, 31)]:
    filtered_df = df[(df["batch_size"] == t) & (df["worker_num"] == w)]
    plt.plot(filtered_df['text_length'], filtered_df[qps], marker='o', linestyle='-', label=f'batch_size={t},worker_num={w}')

plt.title("Text Length vs QPS")
plt.xlabel('Text Length')
plt.ylabel(qps)
# Show the legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))

# (batch_size, worker_num)
for t, w in [(128, 31), (256, 31), (512, 31), (1024, 31)]:
    filtered_df = df[(df["text_length"] == t) & (df["worker_num"] == w)]
    plt.plot(filtered_df['batch_size'], filtered_df[throughput], marker='o', linestyle='-', label=f'text_length={t},worker_num={w}')

plt.title("Batch Size vs Throughput")
plt.xlabel('Batch Size')
plt.ylabel(throughput)
# Show the legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))

# (batch_size, worker_num)
for t, w in [(1, 31), (4, 31), (8, 31), (12, 31), (16, 31)]:
    filtered_df = df[(df["batch_size"] == t) & (df["worker_num"] == w)]
    plt.plot(filtered_df['text_length'], filtered_df[throughput], marker='o', linestyle='-', label=f'batch_size={t},worker_num={w}')

plt.title("Text Length vs Throughput")
plt.xlabel('Text Length')
plt.ylabel(throughput)
# Show the legend
plt.legend()
# Show the plot
plt.show()


In [ ]:
import pandas as pd

csv_file_path = "./db_statistics.csv"

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Define the bins and labels for the aggregation
bins = [-float('inf'), 1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9, 5e9, 1e10, float('inf')]
labels = [
    '< 1e6',
    '< 5e6',
    '< 1e7',
    '< 5e7',
    '< 1e8',
    '< 5e8',
    '< 1e9',
    '< 5e9',
    '< 1e10',
    '>= 1e10'
]

# Create a new column 'category' based on the bins
df['category'] = pd.cut(df['data_num'], bins=bins, labels=labels)

# Aggregate the results by 'category'
result = df['category'].value_counts(sort=False).reset_index()
result.columns = ['Range', 'Count']

result_file = 'aggregated_results.csv'
# Save the result to a new CSV file
result.to_csv(result_file, index=False)

print(f"Aggregated results saved to {result_file}")

In [ ]:
!cat {result_file}

In [ ]:
df.columns

In [ ]:
# Get unique values
unique_text_length = df['category'].unique()
print("Unique values using unique():", unique_text_length)

In [ ]:
data_num_field = "data_num"
total_cost_field = "total_cost"
filtered_df = df[df["category"] == '< 1e6']
df_by_data_num = filtered_df.sort_values(by=data_num_field, ascending=False)
print(df_by_data_num.shape)
df_by_data_num.sort_values(by=total_cost_field, ascending=False).head(n=50)

# AD Recall Secenario

* Data Number: 350万, 64维向量
* Model: hnsw(m=8,cef=120,sef=300), 不做量化, 1分片, 单分片12G(预留了流式更新buffer, 实际6千万), 构建用时 15min
* Service: 1分片23副本, 机器套餐48CPU192G内存, MAX_CPU_UTIL 10% 左右, AVG_CPU_UTIL 3% 左右
* Recall: topk 10000 左右, 单分片候选覆盖数目 20000 - 50000 左右
* QPS: Peak 6K
* Latency Per Shard(p50, p90, p99): 10ms, 14ms, 24ms

# Million Scale

* Data Number: 370万, 128维向量
* Model: hnsw(m=20,cef=400,sef=800), int8量化, 1分片, 单分片2G, 构建用时 10min
* Service: 部署在架构公共资源池, 机器套餐32CPU512G内存, 4副本
* Recall: 召回率 97% 左右, topk 10 左右 (10*1), 单分片候选覆盖数目 12000 - 20000 左右
* QPS: Peak 46K
* Latency Per Shard(p50, p90, p99): 3ms, 3.6ms, 4ms


# 10 Million Scale

* Data Number: 650万, 200维向量
* Model: hnsw(m=20,cef=400,sef=800), int8量化, 16分片, 单分片0.3G, 构建用时 10min
* Service: 部署在架构公共资源池, 机器套餐32CPU512G内存, 4副本
* Recall: 召回率 100% 左右, topk 200左右 (10*16), 单分片候选覆盖数目 100000 左右 (带了过滤条件)
* QPS: Peak 13K
* Latency Per Shard(p50, p90, p99): 5ms, 7.6ms, 10ms


# 50 Million Scale

* Data Number: 3千万, 128维向量
* Model: hnsw(m=24,cef=400,sef=800), int16量化, 10分片, 单分片3G, 构建用时 10min
* Service: 部署在架构公共资源池, 机器套餐32CPU512G内存, 4副本
* Recall: 召回率 94% 左右, topk 10000左右 (1000*10), 单分片候选覆盖数目 15000 - 30000 左右
* QPS: Peak 15K
* Latency Per Shard(p50, p90, p99): 3.5ms, 6ms, 7ms


# 100 Million Scale

* Data Number: 8千万, 256维向量
* Model: hnsw(m=20,cef=400,sef=800), int8量化, 6分片, 单分片16G, 构建用时 0.5h
* Service: 部署在架构公共资源池, 机器套餐32CPU512G内存, 4副本
* Recall: 召回率 92% 左右, topk 60左右 (10*60), 单分片候选覆盖数目 20000 - 26000 左右
* QPS: Peak 57K
* Latency Per Shard(p50, p90, p99): 4ms, 6.5ms, 8ms


# 1000 Million Scale

* Data Number: 49亿, 256维向量
* Model: hnsw(m=20,cef=400,sef=800), int16量化, 30分片, 单分片100G, 构建用时 2h+
* Service: 10分片6副本, 机器套餐64CPU512G内存, MAX_CPU_UTIL 36% 左右, AVG_CPU_UTIL 15% 左右
* Recall: ~~召回率 65% 左右~~, topk 200左右 (20*10), 单分片候选覆盖数目 10000 - 20000 左右
* QPS: Peak 300K
* Latency Per Shard(p50, p90, p99): 10ms, 17ms, 36ms


# 10000 Million Scale

* Data Number: 95亿, 128维向量
* Model: hnsw(m=16,cef=128,sef=100), int8量化, 200分片, 单分片61G, 构建用时 1h
* Service: 50分片7副本, 机器套餐64CPU512G内存, MAX_CPU_UTIL 30% 左右, AVG_CPU_UTIL 10% 左右
* Recall: 召回率 65% 左右, topk 4000左右(20*200), 单分片候选覆盖数目 4000 - 5000 左右
* QPS: Peak 4114K
* Latency Per Shard(p50, p90, p99): <1ms, 1.1ms, 1.6ms


# More

* Data Number: 180亿左右, 128维向量
* Model: hnsw(m=20,cef=400,sef=800), 不使用量化, 64分片, 单分片88G, 构建用时 3h+
* Service: 32分片3副本, 机器套餐32CPU512G内存, MAX_CPU_UTIL 100%, AVG_CPU_UTIL 25% 左右
* Recall: 召回率 98% 左右, topk 1000左右((10-20)*64), 单分片候选覆盖数目 10000 - 17000 左右
* QPS: Peak 74K
* Latency Per Shard(p50, p90, p99): 5ms, 12ms, 48ms